In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install webdriver-manager

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,563 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages 

In [2]:
!sudo chmod +x /path/to/chromedriver

chmod: cannot access '/path/to/chromedriver': No such file or directory


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time
import random

def random_delay(min_delay=5, max_delay=10):
    time.sleep(random.randint(min_delay, max_delay))

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15(KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--remote-debugging-port=9222')
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_driver_path = '/usr/bin/chromedriver'
driver = webdriver.Chrome(options=chrome_options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
        get: () => undefined
    })
    """
})
driver.get("https://lvr.land.moi.gov.tw/")
random_delay()
driver.switch_to.frame(0)
city = Select(driver.find_element(By.NAME, 'city'))
city.select_by_visible_text('臺中市')
random_delay()
region_btn = Select(driver.find_element(By.NAME, "town"))
region_btn.select_by_visible_text("北區")
btn = "#main_form > div:nth-child(2) > div.form-group.mt-0.form-check-inline.qry_general > div > font:nth-child(1) > a"
button = driver.find_elements(By.CSS_SELECTOR, btn)

if len(button) > 0:
    print(button[0].text)
    button[0].click()
random_delay()
random_delay()
table = driver.find_element(By.XPATH, '//*[@id="price_table"]')
rows = table.find_elements(By.TAG_NAME, 'tr')
headers=[]
for row in rows:
    cells = row.find_elements(By.TAG_NAME, 'th')
    headers_data = [cell.text for cell in cells]
    if headers_data:
        headers.append(headers_data)
print(headers)
table_data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, 'td')
    row_data = [cell.text for cell in cells]
    if row_data:
        print(row_data)
        table_data.append(row_data)

df = pd.DataFrame(table_data,columns=headers)
df.to_csv("不動產查詢結果.csv", index=False, encoding='utf-8-sig')
print("資料已儲存為不動產查詢結果.csv")

driver.quit()


搜尋
[['地段位置或門牌', '社區簡稱', '總價(萬元)', '交易日期', '單價\n(萬元/坪)', '總面積\n(坪)', '主建物\n佔比(%)', '型態', '屋齡', '樓別/樓高', '', '', '', '', '', '', '', '', '']]
['北區民族路２４６號', '', '1,860', '113/11/27', '', '88.21', '', '', '43', '全/五層', '', '', '', '', '', '', '', '', '']
['北區福龍街９７號４樓之１０', '', '220', '113/11/26', '25.2', '8.73', '62.23', '', '30', '四層/十二層', '', '', '', '', '', '', '', '', '']
['北區進化路５７８號１０樓之５', '文風鼎盛', '450', '113/11/18', '31.8', '14.15', '70.86', '', '30', '十層/十三層', '', '', '', '', '', '', '', '', '']
['北區國豐街１６巷２０之３號', '', '600', '113/11/15', '19.5', '30.81', '77.87', '', '43', '四層/五層', '', '', '', '', '', '', '', '', '']
['北區文心路四段２００號４樓之５', '龍邦財經大樓', '420', '113/11/14', '18.3', '22.97', '46.93', '', '28', '四層/十三層', '', '', '', '', '', '', '', '', '']
['北區文祥街５號', '', '610', '113/11/12', '29.7', '20.53', '100.00', '', '45', '一層,騎樓/五層', '', '', '', '', '', '', '', '', '']
['北區北屯路１４號８樓之２７', '', '223.3', '113/11/11', '22', '10.15', '100.00', '', '44', '八層/十層', '', '', '', '', '', '', '', '', '